In [4]:
import numpy as np
import heapq
import math
from typing import Callable, TypeVar
from collections import Counter, deque, defaultdict
import itertools
from intervaltree import Interval, IntervalTree
from functools import cmp_to_key, cache, lru_cache
from concurrent.futures import ThreadPoolExecutor
import networkx as nx

np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

T = TypeVar('T')

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

processors = {
  'int_list': lambda x: [int(y) for y in x.split()],
  'int_string': lambda x: [int(y) for y in x]
}

In [47]:
def day1():
    instructions = data(1, lambda x: (x[0], int(x[1:])))
    position, part1, part2 = 50, 0, 0
    for i, v in instructions:
        part2 += v//100
        v = v%100
        current = position + (v if i == 'R' else -v)
        if current >= 100 or (current <= 0 and position):
            part2 += 1
        if not (position := current%100):
            part1 += 1
    return part1, part2

day1()

(1034, 6166)

In [ ]:
tree = IntervalTree()
for x in data(2)[0].split(','):
    start, end = x.split('-')
    tree.addi(int(start), int(end)+1, x)

cap = len(str(tree.end()))//2
result = 0
for i in range(1, 10**cap):
    point = int(str(i)*2)
    if tree.at(point):
        result += point
result

In [47]:
tree = IntervalTree()
for x in data(2)[0].split(','):
    start, end = x.split('-')
    tree.addi(int(start), int(end)+1, x)

cap = len(str(tree.end()))//2
tested = set()
result = 0
for i in range(1, 10**cap):
    digits = str(i)
    if digits in tested:
        continue
    point = digits
    while len(point) < cap*2:
        point += digits
        if point in tested:
            continue
        tested.add(point)
        if tree.at(int(point)):
            # print(point)
            result += int(point)
result

50793864718

In [ ]:
def day2():ranges = data(2)[0].split(',')
    
    return

day2()

NameError: name 'data' is not defined